## 2.2. Komponenty biblioteki LangChain

### Instalacja bibliotek

In [5]:
!pip install -q python-dotenv langchain langchain-openai langchain-community

### Konfiguracja .env i modelu

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś pomocnym asystentem. Odpowiadaj zwięźle."),
    ("user", "Streśc w 1 zdaniu: {tekst}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()  # LCEL: prompt → model → parser
result = chain.invoke({"tekst": "LangChain to biblioteka, która powstała w 2022 roku, w momencie kiedy świat zaczął odkrywać praktyczne możliwości dużych modeli językowych takich jak GPT-5. Jej twórcy zauważyli, że większość projektów opartych na LLM powtarza podobne wzorce – na przykład łączenie promptów w sekwencje, przechowywanie kontekstu rozmowy, wykonywanie zapytań do baz danych oraz wyszukiwarek. Zamiast budować to wszystko od zera, stworzyli bibliotekę, która dostarcza gotowe klocki do składania aplikacji. Dzięki LangChain możemy szybko tworzyć prototypy, a później rozwijać je w kierunku produkcyjnym. Biblioteka stała się swoistym standardem w świecie deweloperów AI, bo pozwala łatwo integrować modele różnych dostawców – nie tylko OpenAI, ale też Anthropic, HuggingFace czy lokalne modele open-source."})
print(result)


LangChain to biblioteka stworzona w 2022 roku, która ułatwia tworzenie aplikacji opartych na dużych modelach językowych, oferując gotowe komponenty do integracji i prototypowania, co czyni ją standardem wśród deweloperów AI.


### Prompt template
ChatPromptTemplate w LangChain to szablon do budowania ustrukturyzowanych promptów dla modeli czatu.
Pozwala definiować role (system, user, assistant), zmienne i format wiadomości, aby w spójny i wielokrotnego użytku sposób generować wejścia dla LLM.

In [3]:
llm = ChatOpenAI(model_name="gpt-4o")
template = """
Specify tags and the main topic of the text.
tags: What are best tags describing text. Give maximum 5 tags separated by comma.
topic: What is the topic of text. Use maximum couple of words

Format response as JSON as below. Return only JSON without markdown tags.
'tags': ['sometag', 'othertag', 'anothertag', 'tag4', 'tag5']
'subject': 'Some subject of text'

text: {input}
"""

prompt_template = ChatPromptTemplate.from_template(template=template)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"input":"They picked a way among the trees, and their ponies plodded along, carefully avoiding the many writhing and interlacing roots.  There was no undergrowth.  The ground was rising steadily, and as they went forward it seemed that the trees became taller, darker, and thicker. There was no sound, except an occasional drip of moisture falling through the still leaves.  For the moment there was no whispering or movement among the branches; but they all got an uncomfortable feeling that they were being watched with disapproval, deepening to dislike and even enmity.  The feeling steadily grew, until they found themselves looking up quickly, or glancing back over their shoulders, as if they expected a sudden blow."})

"{\n    'tags': ['forest', 'suspense', 'mystery', 'nature', 'tension'],\n    'subject': 'Journey through forest'\n}"

### ResponseSchema i OutputParser
ResponseSchema w LangChain to definicja oczekiwanego formatu odpowiedzi (np. pola JSON z nazwą, typem, opisem), która pomaga modelowi zwracać dane w przewidywalnej strukturze.

OutputParser interpretuje surową odpowiedź LLM i przekształca ją w żądaną formę (np. obiekt Pydantic, JSON, lista), ułatwiając dalsze użycie w aplikacji.

In [4]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

tags_schema = ResponseSchema(
    name="tags",
    description=" What are best tags describing text. Give maximum 5 tags separated by comma.",
)
topic_schema = ResponseSchema(
    name="topic",
    description="What is the topic of text. Use maximum couple of words."
)

response_schemas = [tags_schema, topic_schema]
parser = StructuredOutputParser.from_response_schemas(response_schemas)
instructions = parser.get_format_instructions()

template = """
Specify tags and the main topic of the text. Return only JSON without markdown tags.
tags: What are best tags describing text. Give maximum 5 tags separated by comma.
topic: What is the topic of text. Use maximum couple of words

Format response as JSON as below:
'tags': ['sometag', 'othertag', 'anothertag', 'tag4', 'tag5']
'subject': 'Some subject of text'

text: {input}

{instructions}
"""
prompt = ChatPromptTemplate.from_template(template=template)

llm = ChatOpenAI(temperature=0.0)
chain = prompt | llm | parser
output_dict = chain.invoke({
    "input":"They picked a way among the trees, and their ponies plodded along, carefully avoiding the many writhing and interlacing roots.  There was no undergrowth.  The ground was rising steadily, and as they went forward it seemed that the trees became taller, darker, and thicker. There was no sound, except an occasional drip of moisture falling through the still leaves.  For the moment there was no whispering or movement among the branches; but they all got an uncomfortable feeling that they were being watched with disapproval, deepening to dislike and even enmity.  The feeling steadily grew, until they found themselves looking up quickly, or glancing back over their shoulders, as if they expected a sudden blow.",
    "instructions":instructions})
print(output_dict)

{'tags': ['fantasy', 'adventure', 'mystery', 'forest', 'suspense'], 'topic': 'Journey through the forest'}
